In [1]:
import numpy as np
import csv

In [2]:
#file reading
problem_data = []
train_subs = []

with open("train/problem_data.csv") as fproblem:
    reader = csv.reader(fproblem)
    for line in reader:
        problem_data.append(line)
    problem_data = problem_data[1:]
        
with open("train/train_submissions.csv") as ftrain:
    reader = csv.reader(ftrain)
    for line in reader:
        train_subs.append(line)
    train_subs = train_subs[1:]

In [3]:
#imputation of level_type with mode value
for i in problem_data:
    if i[1] == '':
        i[1] = 'A'

In [4]:
#3rd column indicates how difficult was the problem to the user
#similarly, 4th column indicates how easy was the problem to the user

for s, subs in enumerate(train_subs):
    train_subs[s].append(7-int(subs[2]))

In [5]:
tags = set()
for i in [i[3].split(',') for i in problem_data]:
    for j in i:
        tags.add(j)
tags = list(tags)
print(tags)

['', 'geometry', 'divide and conquer', 'combinatorics', 'sortings', 'meet-in-the-middle', 'string suffix structures', 'bitmasks', 'probabilities', 'data structures', '2-sat', 'brute force', 'shortest paths', 'matrices', 'chinese remainder theorem', 'dsu', 'dfs and similar', 'two pointers', 'math', 'constructive algorithms', 'number theory', 'dp', 'graph matchings', 'hashing', 'flows', '*special', 'ternary search', 'expression parsing', 'trees', 'strings', 'graphs', 'games', 'greedy', 'implementation', 'binary search', 'schedules', 'fft']


In [6]:
#unique users extraction
unique_users = list(set(list(map(list, zip(*(train_subs))))[0]))

In [7]:
problem_matrix = np.zeros((len(problem_data), len(tags)))

for p, problem in enumerate(problem_data):
    present_tags = list(problem[3].split(','))
    for t, tag in enumerate(tags):
        if tag in present_tags:
            problem_matrix[p, t] = 1

In [8]:
#previously solved problmes by users
prev_solved = [[] for _ in unique_users]

#(difficulty)user_profile creation
diff_matrix = np.zeros((len(unique_users), len(problem_data), len(tags)))
diff_profile = np.zeros((len(unique_users), len(tags)))

present_tags = []

for u, user in enumerate(unique_users):
    #print('for user :', user)
    for subs in train_subs:
        if subs[0] == user:
            #print('found user in train_subs')
            prev_solved[u].append(subs[1])
            for p, problem in enumerate(problem_data):
                if subs[1] == problem[0]:
                    #print('matching problem :', subs[1])
                    diff_matrix[u, p] = int(subs[2]) * problem_matrix[p]
                    #print(diff_matrix[u, p])
                    break
                    
#user_profile creation
for u, user in enumerate(unique_users):
    diff_profile[u] = diff_matrix[u].sum(axis = 0)
                    
#normalization
for u, user in enumerate(diff_profile):
    denom = sum(user)
    if denom != 0:
        diff_profile[u] = [i/denom for i in user]

In [9]:
#previously solved problmes by users
prev_solved = [[] for _ in unique_users]

#(difficulty)user_profile creation
easy_matrix = np.zeros((len(unique_users), len(problem_data), len(tags)))
easy_profile = np.zeros((len(unique_users), len(tags)))

present_tags = []

for u, user in enumerate(unique_users):
    #print('for user :', user)
    for subs in train_subs:
        if subs[0] == user:
            #print('found user in train_subs')
            prev_solved[u].append(subs[1])
            for p, problem in enumerate(problem_data):
                if subs[1] == problem[0]:
                    #print('matching problem :', subs[1])
                    easy_matrix[u, p] = int(subs[3]) * problem_matrix[p]
                    #print(diff_matrix[u, p])
                    break
                    
#user_profile creation
for u, user in enumerate(unique_users):
    easy_profile[u] = easy_matrix[u].sum(axis = 0)
                    
#normalization
for u, user in enumerate(easy_profile):
    denom = sum(user)
    if denom != 0:
        easy_profile[u] = [i/denom for i in user]

In [11]:
user_query = 'user_' + input('user_id(number) : ')
k  = int(input('number of recommendations required : '))
flag = 1

#profile of queried user
while(flag):
    DorE = input('recommended problems to be easier(E) or difficult(D) : ')
    if DorE == 'D':
        profile = diff_profile
        flag = 0
    elif DorE == 'E':
        profile = easy_profile
        flag = 0
        
recommendation = []

for p, problem in enumerate(problem_data):
    user_query_index = unique_users.index(user_query)
    if problem[0] not in prev_solved[user_query_index]:
        recommendation.append((problem[0], np.sum((profile[user_query_index]) * (problem_matrix[p]))))
    
#profile sorted acording to their dot product values
recommendation = [index for index, value in sorted(recommendation, reverse=True, key=lambda x:x[1])]

#final recommendation list calculation based on value of k
recommendation = recommendation[:k]
print('\nrecommended problems : ', recommendation)

user_id(number) : 3311
number of recommendations required : 5
recommended problems to be easier(E) or difficult(D) : E

recommended problems :  ['prob_2691', 'prob_5664', 'prob_1991', 'prob_5425', 'prob_5596']
